# Criminal prediction using Face Recognition


Now-a-days most people in this modern age has atleast one vehicle in their homes. The owners are in fear of having their vehicles being stolen from outside of their home.In image analysis,Face recognition is one of the important and successful method.The concept of Face recognition can be applied to vehicle security.This can be done by recognizing the facial features of the authorized person to unlock the engines.In case of any theft, the system will not let engines start.

In [13]:
#MongoDB is a noSQL database which used to store huge amount Data with key value pairs
import dlib
import numpy as np
import face_recognition
import cv2
import os
import pymongo
import datetime
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]
mycol = mydb["attendance"]


#Note:

The images of the authenticated people who can access the vehicle are stored in the folder called hs. The below code collects the images with their image names from the hs folder for example owner jack image with name jack.jpg. Images are stored in images list and their Labels are stored in classnames list.

Histogram of oriented gradiants(HOG) classifier:- 

They find the facial landmarks of the tilted images and make the image look straight with the use of D-lib library at the backend . After this process they send this image to the neural network that gives us the encoded features.

In [4]:

images=[]
classnames=[]
path='hs/'
mylist=os.listdir(path)
print(mylist)
for cl in mylist:
    curimage=cv2.imread(f'{path}/{cl}')
    images.append(curimage)                    
    classnames.append(os.path.splitext(cl)[0])                   
print(classnames) 
def findencodings(images):
    encodings=[]
    for img in images:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        faceloc=face_recognition.face_locations(img)[0]
        encodehar=face_recognition.face_encodings(img)[0]
        encodings.append(encodehar)
    return encodings
encodelistknown=findencodings(images)
print('encodings complete')
cap=cv2.VideoCapture(0)
flag=True
c=0
while(flag==True):
    success,img=cap.read()
    #img=cv2.resize(img,(0,0),None,0.25,0.25)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    facelocq=face_recognition.face_locations(img)
    encodeharq=face_recognition.face_encodings(img,facelocq)
    for enc,floc in zip(encodeharq,facelocq):
    
        results=face_recognition.compare_faces(encodelistknown,enc)
        dresults=face_recognition.face_distance(encodelistknown,enc)
        print(dresults)
        matchind=np.argmin(dresults)
        if results[matchind]:
            name=classnames[matchind]
            print(name)
            y1,x2,y2,x1=floc
            y1,x2,y2,x1=y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(floc[3],floc[0]),(floc[1],floc[2]),(0,255,0),2)
            #cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(25,25),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            s=mycol.find_one({})
            now=datetime.datetime.now()
            dtstring=now.strftime('%H:%M:%S')
            mydict = { "name": name, "date":dtstring}
            mycol.insert_one(mydict)
            cv2.imshow('frame',img)
            c=c+1
        if c==2:
            flag=False
# When everything done, release the capture 
cap.release() 
cv2.destroyAllWindows()



['harish.jpg', 'himg.jpg', 'suriya.jpg']
['harish', 'himg', 'suriya']
encodings complete
[0.58169043 0.89700895 0.47714234]
suriya
[0.59591002 0.92423643 0.50875631]
suriya


In [15]:
#Demo using sample Image 

img1=face_recognition.load_image_file('hs/suriya.jpeg')
img1=cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
faceloc=face_recognition.face_locations(img1)[0]
print(faceloc)
encodehar=face_recognition.face_encodings(img1)[0]
results=face_recognition.compare_faces(encodelistknown,encodehar)
dresults=face_recognition.face_distance(encodelistknown,encodehar)
#cv2.putText(img2,f'{results}{round(dresults[0],2)}',(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,255))
print(dresults)
matchind=np.argmin(dresults)
if results[matchind]:
    name=classnames[matchind].upper()
    print(name)
    cv2.rectangle(img1,(faceloc[3],faceloc[0]),(faceloc[1],faceloc[2]),(255,0,255),2)
    cv2.putText(img1,f'{name}{round(dresults[0],2)}',(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,255))
cv2.imshow('image',img1)
cv2.waitKey(0)  
cv2.rectangle(img1,(faceloc[3],faceloc[0]),(faceloc[1],faceloc[2]),(255,0,255),2)
img2=face_recognition.load_image_file('hs/himg.jpg')
img2=cv2.cvtColor(img2,cv2.COLOR_BGR2RGB)
faceloc1=face_recognition.face_locations(img2)[0]
print(faceloc1)
encodehar1=face_recognition.face_encodings(img2)[0]
cv2.rectangle(img2,(faceloc1[3],faceloc1[0]),(faceloc1[1],faceloc1[2]),(255,0,255),2)
#cv2.imshow('image',img1)
#cv2.imshow('image',img2)
results=face_recognition.compare_faces([encodehar],encodehar1)
dresults=face_recognition.face_distance([encodehar],encodehar1)
cv2.putText(img2,f'{results}{round(dresults[0],2)}',(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,255))
print(results)
cv2.imshow('image',img2)
cv2.waitKey(0)

(384, 633, 705, 312)
[0.57941281 0.88289557 0.26852943]
SURIYA
(44, 135, 95, 83)
[False]


-1

In [7]:
#Displaying the Owners List detected from the database
s=mycol.find({})
for x in s:
    print(x)

{'_id': ObjectId('60902e2cdcfce39041e5a65e'), 'name': 'suriya', 'date': '22:39:00'}
{'_id': ObjectId('60902e2ddcfce39041e5a65f'), 'name': 'suriya', 'date': '22:39:01'}
